<a href="https://colab.research.google.com/github/Adwait2302/python_proj1/blob/main/SY_positions_daily.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Importing required libraries, loading excel files and getting the date.

In [ ]:
import pandas as pd
from datetime import datetime

today = datetime.today().strftime('%d-%m-%y')

cme_file = 'SY Positions(CME).xlsx'
ice_file = 'SY Positions(ICE).xlsx'

Preparing the tables for CME product CL, HO, RB. Creating an empty dictionary and updating it with the net outright positions for all the products.

In [ ]:
acc_data = pd.read_excel(cme_file, sheet_name='Account wise')
acc_data = acc_data.drop(columns=['Firm Id',
                                  'CC Code',
                                  'Put / Call',
                                  'Strike',
                                  'Underlying Period Code'])

prod_name = ['CL', 'HO', 'RB', 'NG']

expiry_code = {
      'CL' : ['202504', '202505' ,'202506', '202507', '202508', '202509', '202510', '202511', '202512',
              '202601', '202602', '202603', '202604', '202605', '202606', '202607', '202608', '202609', '202610', '202611', '202612',
              '202701', '202702', '202703', '202704', '202705', '202706', '202707', '202708', '202709', '202710', '202711', '202712',
              '202801', '202802', '202803', '202804', '202805', '202806', '202807', '202808', '202809', '202810', '202811', '202812',
              '202901', '202902', '202903', '202904', '202905', '202906', '202907', '202908', '202909', '202910', '202911', '202912',
              '203001', '203002', '203003', '203004', '203005', '203006', '203007', '203008', '203009', '203010', '203011', '203012',
              '203101', '203102', '203103', '203104', '203105', '203106', '203107', '203108', '203109', '203110', '203111', '203112',
              '203201', '203202', '203203', '203204', '203205', '203206', '203207', '203208', '203209', '203210', '203211', '203212',
              '203301', '203302', '203303'],
      'RB' : ['202504', '202505' ,'202506', '202507', '202508', '202509', '202510', '202511', '202512',
              '202601', '202602', '202603', '202604', '202605', '202606', '202607', '202608', '202609', '202610', '202611', '202612',
              '202701'],
      'HO' : ['202504', '202505' ,'202506', '202507', '202508', '202509', '202510', '202511', '202512',
              '202601', '202602', '202603', '202604', '202605', '202606', '202607', '202608', '202609', '202610', '202611', '202612',
              '202701', '202702', '202703', '202704', '202705', '202706'],
      'NG' : ['202504', '202505' ,'202506', '202507', '202508', '202509', '202510', '202511', '202512',
              '202601', '202602', '202603', '202604', '202605', '202606', '202607', '202608', '202609', '202610', '202611', '202612',
              '202701', '202702', '202703', '202704', '202705', '202706']
}

net_outright = {}

for prod in prod_name:
  if prod != 'NG':
    df = acc_data[acc_data['Ticker Symbol'].str[0:2].isin([f'{prod}'])]

    expiry_df = pd.DataFrame({"Period Code" : expiry_code[prod]})
    expiry_df['Period Code'] = expiry_df['Period Code'].astype(int)

    pivot = df.pivot_table(index='Period Code', columns='Acct Id',
                                  values='Net Positions', aggfunc='sum')
    merged_pivot = pd.merge(expiry_df, pivot, left_on='Period Code', right_index=True, how='left')

    pivot = merged_pivot.fillna(0)
    pivot.set_index('Period Code', inplace=True)
    pivot = pivot.T

    net_outright[prod] = pivot.values.sum()
    pivot.iloc[:,0] = pivot.iloc[:,0] - pivot.sum(axis=1)

    spread = pivot
    for i in range(1, pivot.shape[1]):
      for j in range(pivot.shape[0]):
        spread.iloc[j, i] = pivot.iloc[j, i]+pivot.iloc[j, i-1]     #Converting outright data into spread data

    row_total = spread.sum(axis=1)
    grand_total_col = {'Grand Total' : row_total}
    spread = pd.concat([pd.DataFrame(grand_total_col), spread], axis=1)     #Adding grand total row to the front

    spread.loc["Grand Total"] = spread.sum(axis=0)      # Compute column sums and insert as first row
    spread = spread.iloc[[-1] + list(range(len(spread) - 1))]     # Move the new row to the top

    sheet_name = f"{prod}"  # Create dynamic sheet name

    globals()[f'{prod}_df'] = spread.copy()

  else:

    df = acc_data[acc_data['Ticker Symbol'].str[0:2].isin(['NG', 'HH'])]

    expiry_df = pd.DataFrame({"Period Code" : expiry_code[prod]})
    expiry_df['Period Code'] = expiry_df['Period Code'].astype(int)

    pivot = df.pivot_table(index='Period Code', columns='Acct Id',
                                  values='Net Positions', aggfunc='sum')
    merged_pivot = pd.merge(expiry_df, pivot, left_on='Period Code', right_index=True, how='left')

    pivot = merged_pivot.fillna(0)
    pivot.set_index('Period Code', inplace=True)
    pivot = pivot.T

    net_outright[prod] = pivot.values.sum()
    pivot.iloc[:,0] = pivot.iloc[:,0] - pivot.sum(axis=1)

    spread = pivot
    for i in range(1, pivot.shape[1]):
      for j in range(pivot.shape[0]):
        spread.iloc[j, i] = pivot.iloc[j, i]+pivot.iloc[j, i-1]     #Converting outright data into spread data

    row_total = spread.sum(axis=1)
    grand_total_col = {'Grand Total' : row_total}
    spread = pd.concat([pd.DataFrame(grand_total_col), spread], axis=1)     #Adding grand total row to the front

    spread.loc["Grand Total"] = spread.sum(axis=0)      # Compute column sums and insert as first row
    spread = spread.iloc[[-1] + list(range(len(spread) - 1))]     # Move the new row to the top

    sheet_name = f"{prod}"  # Create dynamic sheet name

    globals()[f'{prod}_df'] = spread.copy()


Preparing files for ICE products G, SB, KC, CC, C.

In [ ]:
acc_data = pd.read_excel(ice_file, sheet_name='Account wise')
acc_data = acc_data[acc_data['Security Type'] == 'FUT'].copy()
acc_data['Expiry Date'] = acc_data['Expiry Date'].astype(str).str[:6].astype(int)

acc_data = acc_data.drop(columns=['Exchange Code',
                                  'PutOrCall',
                                  'Strike Price',
                                  'Outright Margin',
                                  'Position Type',
                                  'Regime',
                                  'Customer Type',
                                  'Account type'])

prod_name = ['G', 'SB', 'KC', 'CC', 'C']

### Make sure to update the expiries before running the code ###

contract_months = {
    'G' : ['202503', '202504', '202505' ,'202506', '202507', '202508', '202509', '202510', '202511', '202512',
           '202601', '202602', '202603', '202604', '202605', '202606', '202607', '202608', '202609', '202610', '202611', '202612',
           '202701', '202702', '202703', '202704', '202705', '202706', '202707', '202708', '202709', '202710', '202711', '202712',
           '202801', '202802', '202803', '202804', '202805', '202806', '202807', '202808', '202809', '202810', '202811', '202812',],
    'C' : ['202503', '202505', '202507', '202509', '202512', '202603', '202605', '202607', '202609', '202612'],
    'CC' : ['202503', '202505', '202507', '202509', '202512', '202603', '202605', '202607', '202609', '202612'],
    'KC' : ['202503', '202505', '202507', '202509', '202512', '202603', '202605', '202607', '202609', '202612', '202703', '202705', '202707', '202709', '202712'],
    'SB' : ['202505', '202507', '202510', '202603', '202605', '202607', '202610', '202703', '202705', '202707', '202710']
}

for prod in prod_name:
  expiry_df = pd.DataFrame({"Expiry Date" : contract_months[prod]})
  expiry_df['Expiry Date'] = expiry_df['Expiry Date'].astype(int)

  df = acc_data[acc_data['Exchange Contract Code'] == prod]

  pivot = df.pivot_table(index='Expiry Date', columns='Portfolio Name',
                            values='Net Position', aggfunc='sum')
  merged_pivot = pd.merge(expiry_df, pivot, left_on='Expiry Date', right_index=True, how='left')

  pivot = merged_pivot.fillna(0)
  pivot.set_index('Expiry Date', inplace=True)
  pivot = pivot.T

  try:
    df = pivot.drop(index=['SY004', 'SY167', 'SY198'])
  except:
    df = pivot

  net_outright[prod] = df.values.sum()
  df.iloc[:,0] = df.iloc[:,0]-df.sum(axis=1)      #adjusting for the net ouright

  spread = df
  for i in range(1, df.shape[1]):
    for j in range(df.shape[0]):
      spread.iloc[j, i] = df.iloc[j, i]+df.iloc[j, i-1]     #Converting outright data into spread data

  row_total = spread.sum(axis=1)
  grand_total_col = {'Grand Total' : row_total}
  spread = pd.concat([pd.DataFrame(grand_total_col), spread], axis=1)     #Adding grand total row to the front

  spread.loc["Grand Total"] = spread.sum(axis=0)      # Compute column sums and insert as first row
  spread = spread.iloc[[-1] + list(range(len(spread) - 1))]     # Move the new row to the top

  sheet_name = f"{prod}"  # Create dynamic sheet name

  globals()[f'{prod}_df'] = spread.copy()

Gets the BZ data from CME exchange and B data from ICE and creates a combined dataframe containing total Brent positions data.

In [ ]:
expiry_code = {
    'B' : ['202505' ,'202506', '202507', '202508', '202509', '202510', '202511', '202512',
            '202601', '202602', '202603', '202604', '202605', '202606', '202607', '202608', '202609', '202610', '202611', '202612',
            '202701', '202702', '202703', '202704', '202705', '202706', '202707', '202708', '202709', '202710', '202711', '202712',
            '202801', '202802', '202803', '202804', '202805', '202806', '202807', '202808', '202809', '202810', '202811', '202812',
            '202901', '202902', '202903', '202904', '202905', '202906', '202907', '202908', '202909', '202910', '202911', '202912',
            '203001', '203002', '203003', '203004', '203005', '203006', '203007', '203008', '203009', '203010', '203011', '203012',
            '203101', '203102', '203103', '203104', '203105', '203106', '203107', '203108', '203109', '203110', '203111', '203112',
            '203201', '203202', '203203', '203204', '203205', '203206', '203207', '203208', '203209', '203210', '203211', '203212',
            '203301', '203302', '203303']
}

acc_data = pd.read_excel(cme_file, sheet_name='Account wise')
acc_data = acc_data.drop(columns=['Firm Id',
                                  'CC Code',
                                  'Put / Call',
                                  'Strike',
                                  'Underlying Period Code'])

df = acc_data[acc_data['Ticker Symbol'].str[0:2].isin(['BZ'])]

expiry_df = pd.DataFrame({"Period Code" : expiry_code['B']})
expiry_df['Period Code'] = expiry_df['Period Code'].astype(int)

pivot = df.pivot_table(index='Period Code', columns='Acct Id',
                              values='Net Positions', aggfunc='sum')
merged_pivot = pd.merge(expiry_df, pivot, left_on='Period Code', right_index=True, how='left')

acc_data = pd.read_excel(ice_file, sheet_name='Account wise')
acc_data = acc_data[acc_data['Security Type'] == 'FUT'].copy()
acc_data['Expiry Date'] = acc_data['Expiry Date'].astype(str).str[:6].astype(int)

acc_data = acc_data.drop(columns=['Exchange Code',
                                  'PutOrCall',
                                  'Strike Price',
                                  'Outright Margin',
                                  'Position Type',
                                  'Regime',
                                  'Customer Type',
                                  'Account type'])

df = acc_data[acc_data['Exchange Contract Code'] == 'B']

pivot = df.pivot_table(index='Expiry Date', columns='Portfolio Name',
                          values='Net Position', aggfunc='sum')
pivot.rename_axis("Period Code", inplace=True)
merged_pivot = pd.merge(merged_pivot, pivot, left_on='Period Code', right_index=True, how='left')

pivot = merged_pivot.fillna(0)
pivot.set_index('Period Code', inplace=True)
pivot = pivot.T

try:
    df = pivot.drop(index=['SY004', 'SY167', 'SY198'])
except:
  df = pivot

net_outright['B'] = df.values.sum()
df.iloc[:,0] = df.iloc[:,0] -df .sum(axis=1)      #adjusting for the net ouright

spread = df
for i in range(1, df.shape[1]):
  for j in range(df.shape[0]):
    spread.iloc[j, i] = df.iloc[j, i]+df.iloc[j, i-1]     #Converting outright data into spread data

row_total = spread.sum(axis=1)
grand_total_col = {'Grand Total' : row_total}
spread = pd.concat([pd.DataFrame(grand_total_col), spread], axis=1)     #Adding grand total row to the front

spread.loc["Grand Total"] = spread.sum(axis=0)      # Compute column sums and insert as first row
spread = spread.iloc[[-1] + list(range(len(spread) - 1))]     # Move the new row to the top

B_df = spread

Getting net outright and net spread positions.

In [ ]:
net_spread = {}

df_dict = {
      'CL' : CL_df,
      'B' : B_df,
      'G': G_df,
      'HO' : HO_df,
      'RB' : RB_df,
      'NG' : NG_df,
      'SB': SB_df,
      'KC': KC_df,
      'CC': CC_df,
      'C': C_df
}

for key in net_outright.keys():
    if key in df_dict:  # Ensure the key exists in the dictionary
        net_spread[key] = df_dict[key].iloc[0, 0]  # Access first row, first column

net_pos = pd.DataFrame({key: (net_outright[key], net_spread[key]) for key in net_outright}).T
net_pos.columns = ['Net Outright', 'Net Spread']
net_pos = net_pos.reindex(['CL', 'B', 'G', 'HO', 'RB', 'NG', 'SB', 'KC', 'CC', 'C'])

net_pos

Downloading the dataframes into excel file

In [ ]:
file_name = f'Futures Positions {today}.xlsx'

with pd.ExcelWriter(file_name, engine='openpyxl') as writer:
  for prod, df in df_dict.items():
      df.to_excel(writer, sheet_name=f"{prod}", index=True)